In [1]:
import torch
import torch.nn as nn
import torchmetrics

from datetime import date

import sys; sys.path.append("../")
from models.model_ViT import vit_mse_losses, autoencoderViT

from utils import (
    load_data,
    training_loop,
    TiledMSE,
    data_protocol_bd
)


In [2]:
# define training hyper-parameters 
LEARNING_RATE = 0.0001
NUM_EPOCHS = 500
BATCH_SIZE = 64
num_workers = 6
augmentations = False

In [3]:
# define dataset folder 
DATA_FOLDER = '/home/lcamilleri/data/s12_buildings/data_patches/'

In [4]:
# define model & criterion
model = autoencoderViT(chw=(10, 64, 64), n_patches=4, n_blocks=1, hidden_d=768, n_heads=12, decoder_n_blocks=1, decoder_hidden_d=512, decoder_n_heads=16)
criterion = vit_mse_losses(n_patches=4)
lr_scheduler = 'reduce_on_plateau'

/home/lcamilleri/git_repos/Phileo-downstream-tasks/models/model_ViT.py:139: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embeddings(self.n_patches ** 2 + 1,
/home/lcamilleri/git_repos/Phileo-downstream-tasks/models/model_ViT.py:185: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embed = nn.Parameter(torch.tensor(get_positional_embeddings(self.n_patches ** 2 + 1,


In [5]:
# define save folder location
NAME = model.__class__.__name__
OUTPUT_FOLDER = f'trained_models/{date.today().strftime("%d%m%Y")}_{NAME}_aug={augmentations}'
if lr_scheduler is not None:
    OUTPUT_FOLDER = f'trained_models/{date.today().strftime("%d%m%Y")}_{NAME}_aug={augmentations}_{lr_scheduler}'
    if lr_scheduler == 'reduce_on_plateau':
        LEARNING_RATE = LEARNING_RATE / 100000 # for warmup start

In [6]:
# attach model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# create a dataset
# split_percentage defines the % of samples taken from 
# each specified region ... example split_percentage = 0.2 
# will create a training set consisting of only 20% of the 
# images from each region.

x_train, y_train, x_val, y_val, x_test, y_test = data_protocol_bd.protocol_split(folder=DATA_FOLDER,
                                                                                     split_percentage=1)


In [8]:
# create dataloaders for training
dl_train, dl_val, dl_test = load_data(x_train, y_train, x_val, y_val, x_test, y_test,
                                      with_augmentations=augmentations,
                                      num_workers=num_workers,
                                      batch_size=BATCH_SIZE,
                                      encoder_only=False,
                                      )

In [9]:
# define some torch metrics
# wmape = torchmetrics.WeightedMeanAbsolutePercentageError(); wmape.__name__ = "wmape"
# mae = torchmetrics.MeanAbsoluteError(); mae.__name__ = "mae"
# mse = torchmetrics.MeanSquaredError(); mse.__name__ = "mse"

In [ ]:
# run training loop
training_loop(
        num_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE,
        model=model,
        criterion=criterion,
        device=device,
        # metrics=[
        #     mse.to(device),
        #     wmape.to(device),
        #     mae.to(device),
        # ],
        metrics=[],
        lr_scheduler=lr_scheduler,
        train_loader=dl_train,
        val_loader=dl_val,
        test_loader=dl_test,
        name=NAME,
        out_folder=OUTPUT_FOLDER,
        predict_func=None,
    )

Starting training...



Epoch 5/500: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 598/598 [13:00<00:00,  1.31s/it, loss=0.0755, val_loss=0.0650, lr=1e-5]


Warmup finished


Epoch 303/500:   0%|                                                                                                                                                                                             | 0/598 [00:00<?, ?it/s]

In [ ]:
from IPython.display import display, Image
display(Image(filename= f'{OUTPUT_FOLDER}/visualisations/test_pred.png'))
display(Image(filename= f'{OUTPUT_FOLDER}/loss.png'))
display(Image(filename= f'{OUTPUT_FOLDER}/lr.png'))